In [1]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
import re

print("Libraries imported successfully")

Libraries imported successfully


In [2]:
data = pd.read_csv("preprocessed_dataset.csv")

# Verification
print(f"Dataset shape: {data.shape}")
print("Class distribution:\n", data["label"].value_counts().head(5), "...", data["label"].value_counts().tail(5))
print("Sample rows:\n", data.head())

Dataset shape: (477, 2)
Class distribution:
 label
Dengue         48
Typhoid        43
Chicken pox    36
allergy        28
Psoriasis      25
Name: count, dtype: int64 ... label
Hypertension             10
Dimorphic Hemorrhoids     9
Migraine                  7
Acne                      5
Arthritis                 4
Name: count, dtype: int64
Sample rows:
        label                      extracted_symptoms
0  Psoriasis  dry scaly patches itchy rash skin rash
1  Psoriasis              peeling stinging sensation
2  Psoriasis                         joint pain pain
3  Psoriasis                     silver like dusting
4  Psoriasis                                     NaN


In [4]:
bio_word_vec = KeyedVectors.load_word2vec_format(r"C:\Users\ACER\Downloads\BioWordVec_PubMed_MIMICIII_d200.vec.bin", binary=True)
print(f"Loaded embeddings with {len(bio_word_vec)} terms")
print("Sample check: 'fever' in vocab?", "fever" in bio_word_vec)

Loaded embeddings with 16545452 terms
Sample check: 'fever' in vocab? True


In [5]:
def symptoms_to_embedding(symptoms, model):
    symptom_list = symptoms.split()
    vectors = []
    for symptom in symptom_list:
        words = symptom.split()
        word_vecs = [model[word] for word in words if word in model]
        if word_vecs:
            vectors.append(np.mean(word_vecs, axis=0))
    return np.mean(vectors, axis=0) if vectors else np.zeros(200)

data["symptom_embedding"] = data["extracted_symptoms"].apply(lambda x: symptoms_to_embedding(x, bio_word_vec))
X = np.vstack(data["symptom_embedding"].values)
y = data["label"].values

print(f"X shape: {X.shape}")
print("Sample embedding (first 5 dims):", X[0][:5])

AttributeError: 'float' object has no attribute 'split'